# Regression Model A: Field 2

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 2 (F2)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E2" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E2" ),]
dE4=d4[which(d4$Estate=="E2" ),]
dE5=d5[which(d5$Estate=="E2" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Train and test data

Split the dataset into train and test sets. 

After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

We use the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 

In [7]:
train = dg1[dg1$year<=2016,]
test = dg1[dg1$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [8]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-56.938 -18.002  -2.504  16.680  57.995 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
(Intercept)    7.609e+01  2.514e+01   3.027  0.00599 **
rainfall       3.458e-02  3.266e-02   1.059  0.30070   
foliar        -3.261e+00  1.928e+00  -1.691  0.10429   
prun.time      7.926e+00  5.271e+00   1.504  0.14621   
prun.cost      3.806e-03  7.249e-03   0.525  0.60458   
manuring.time  3.833e+00  1.270e+01   0.302  0.76557   
manuring.cost -8.376e-05  7.377e-04  -0.114  0.91059   
pd.time        2.491e+01  1.030e+01   2.418  0.02391 * 
pd.cost       -6.453e-03  7.487e-03  -0.862  0.39759   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 34.78 on 23 degrees of freedom
Multiple R-squared:  0.4038,	Adjusted R-squared:  0.1964 
F-stati

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [9]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,3718.0460,3718.0460,3.07325263,0.09291070
foliar,1,1519.0578,1519.0578,1.25561878,0.27404654
prun.time,1,5167.9011,5167.9011,4.27167002,0.05018932
prun.cost,1,233.1285,233.1285,0.19269872,0.66477620
manuring.time,1,422.3260,422.3260,0.34908510,0.56039330
manuring.cost,1,100.1535,100.1535,0.08278459,0.77613533
pd.time,1,6784.6306,6784.6306,5.60802200,0.02666252
pd.cost,1,898.9293,898.9293,0.74303456,0.39758908
Residuals,23,27825.5870,1209.8081,NA,NA


### Using stepwise

In [10]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=234.58
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.cost  1      15.6 27841 232.59
- manuring.time  1     110.1 27936 232.70
- prun.cost      1     333.5 28159 232.96
- pd.cost        1     898.9 28725 233.59
- rainfall       1    1356.2 29182 234.10
<none>                       27826 234.57
- prun.time      1    2736.3 30562 235.58
- foliar         1    3460.5 31286 236.33
- pd.time        1    7075.7 34901 239.82

Step:  AIC=232.59
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.time  1      96.6 27938 230.70
- prun.cost      1     329.6 28171 230.97
- pd.cost        1     911.5 28753 231.62
- rainfall       1    1510.1 29351 232.28
<none>                       27841 232.59
- prun.time      1    3267.3 31108 234.14
- foliar         1    3479.6 31321 234.36
- pd.t


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + pd.time, 
    data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-58.043 -21.237  -2.447  19.837  56.217 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept) 77.46338   22.95215   3.375  0.00225 **
rainfall     0.04034    0.02661   1.516  0.14122   
foliar      -2.91786    1.78648  -1.633  0.11401   
prun.time    9.52504    3.87649   2.457  0.02072 * 
pd.time     19.72540    7.85082   2.513  0.01826 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 32.99 on 27 degrees of freedom
Multiple R-squared:  0.3702,	Adjusted R-squared:  0.2769 
F-statistic: 3.968 on 4 and 27 DF,  p-value: 0.01167


 rainfall    foliar prun.time   pd.time 
 1.083565  1.031967  1.072826  1.068669 



Call:
lm(formula = yield ~ rainfall + foliar + prun.time + pd.time, 
    data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-58.043 -21.237  -2.447  19.837  56.217 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept) 77.46338   22.95215   3.375  0.00225 **
rainfall     0.04034    0.02661   1.516  0.14122   
foliar      -2.91786    1.78648  -1.633  0.11401   
prun.time    9.52504    3.87649   2.457  0.02072 * 
pd.time     19.72540    7.85082   2.513  0.01826 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 32.99 on 27 degrees of freedom
Multiple R-squared:  0.3702,	Adjusted R-squared:  0.2769 
F-statistic: 3.968 on 4 and 27 DF,  p-value: 0.01167


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of the model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [11]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,107.799,100.79116
34,61.336,109.04720
35,62.955,86.16192
36,55.218,112.64990
37,49.703,137.04245
38,75.186,117.78801
39,164.932,116.90718
40,256.238,105.45750
41,158.787,109.64396


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (R-square) to know how close the data to the fitted regression line. 

In [12]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(a1)$r.squared)
fit

RMSE         R2 
76.4782304  0.4037769